In [1]:
import os
from langchain.document_loaders import DirectoryLoader ,PyPDFLoader
from langchain.llms import VertexAI
from langchain import LLMChain

from langchain.text_splitter import RecursiveCharacterTextSplitter
from IPython.core.display import Markdown
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate


# os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
# os.environ["WANDB_PROJECT"] = "langchain-tracing"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'ghc-001-7755ed5582db.json'


In [105]:
directory = "media/resumes/"
dl = DirectoryLoader(directory, "**/*.pdf")
docs = dl.load()


In [106]:

splitter = RecursiveCharacterTextSplitter(chunk_size=1400, chunk_overlap=100)
document_section = splitter.split_documents(docs)



In [107]:
embeddings = VertexAIEmbeddings()

db = Chroma.from_documents(document_section, embeddings)


In [108]:
retriever = db.as_retriever(search_kwargs = dict(k=5))


In [109]:
question = "Create a json for all the people in this resumes"
docs = retriever.get_relevant_documents(question)
docs

[Document(page_content='Rufus Stewart A C C O U N T E X E C U T I V E\n\nPERSONAL PROFILE\n\nWith over 10 years experience, my current role involves liaising with clients, planning, organising events, advertising, PR and research.\n\nI specialise in building lasting client relationships and developing\n\neffective strategies. With over 10 years experience, my role involves\n\nliaising with clients, planning, organising events, advertising, PR and\n\nresearch. I specialise in building lasting client relationships and\n\ndeveloping effective strategies.\n\nWORK EXPERIENCE\n\nSKILLS\n\n2011 - 2015\n\nAccount Executive\n\nStrategy development\n\nStudio Shodwe\n\nOrganisation\n\nPublic relations\n\n2016 - 2018\n\nAccount Executive\n\nCustomer service & relations\n\nThynk Unlimited\n\nInnovative thinking\n\nExcellent planning skills\n\n2019 - 2021\n\nAccount Executive\n\nNegotiation & networking\n\nTimmerman Industries\n\nAbility to multi-task\n\nWork to deadlines\n\nEDUCATION\n\nCONTACT\n\n

In [134]:
"Name": "John Doe",
    "Summary": "Software Engineer with 10 years of experience in developing scalable web applications. Proficient in Java, Python, C++.",
    "Work Experience": "Google Search, Facebook Ads, Uber",
    "Years Of Experience": "10 years",
    "Skills": ["Java", "Python", "C++"],
    "Projects": ["Google Search", "Facebook Ads", "Uber"],
    "Education": "Stanford University",
    "Certifications": ["AWS Certified Solutions Architect - Associate", "Google Cloud Certified Professional Cloud Architect"],
    "Awards and Achievements": ["Google Founders' Award", "ACM Grace Hopper Award"]


SyntaxError: illegal target for annotation (1452720814.py, line 1)

In [3]:
prompt_template = """
In this task, you'll be analyzing the resume of a job candidate. Your objective is to extract key information from all the resume of these people

Here are candidate's resume:

***

{resume}

***

Your task is to carefully read through the resume and identify key information about the candidate. This includes:

- Name
- Summary of their professional profile
- Work experience
- Years of experience
- Skills
- Major projects they've worked on
- Education background
- Certifications
- Awards and achievements

For example, if you're examining the resume of a software engineer named John Doe, your output should be a markdown table:
Create a table and fill the respective details from the resume inside it and in Markdown format and it should include the following fields:

Name | Summary | Work Experience | Years Of Experience | Skills | Projects | Education | Certifications | Awards and Achievements
John Doe | Software Engineer with 10 years of experience in developing scalable web applications. Proficient in Java, Python, C++. | Google Search, Facebook Ads, Uber | 10 years | Java, Python, C++ | Google Search, Facebook Ads, Uber | Stanford University | AWS Certified Solutions Architect - Associate, Google Cloud Certified Professional Cloud Architect | Google Founders' Award, ACM Grace Hopper Award

When you encounter a category for which the resume provides no information, please use None as the value.

The quality of your work will be assessed based on:

The completeness and accuracy of the information you extract from the resume.
The relevance of the extracted information to the job posting.
The correctness of the JSON format in your output.
Now, here's your question:

Question: {question}

When you've completed your analysis, please provide the extracted details in Markdown format.
"""

In [172]:
prompt_template

"\nIn this task, you'll be analyzing the resume of a job candidate. Your objective is to extract key information from all the resume of these people\n\nHere are candidate's resume:\n\n***\n\n{resume}\n\n***\n\nYour task is to carefully read through the resume and identify key information about the candidate. This includes:\n\n- Name\n- Summary of their professional profile\n- Work experience\n- Years of experience\n- Skills\n- Major projects they've worked on\n- Education background\n- Certifications\n- Awards and achievements\n\nFor example, if you're examining the resume of a software engineer named John Doe, your output should be a markdown table:\nCreate a table and fill the respective details from the resume inside it and in Markdown format and it should include the following fields:\n\nName | Summary | Work Experience | Years Of Experience | Skills | Projects | Education | Certifications | Awards and Achievements\nJohn Doe | Software Engineer with 10 years of experience in develo

In [4]:

prompt = PromptTemplate(template=prompt_template, input_variables=["resume" , "question"])
loader1 = PyPDFLoader('media/resumes/Black_and_White_Minimalist_Graphic_Designer_Resume.pdf')
pages = loader1.load()

# Get the content of the resume
indiv_resume = pages[0].page_content






llm = VertexAI(model_name="text-bison@001") # select the model
llm_chain = LLMChain(prompt=prompt, llm=llm)
resume = "\n\n".join([doc.page_content for doc in docs[:2]])
output = Markdown(llm_chain.run(resume = resume ,  question="Create a table for Rufus Stewart"))

NameError: name 'docs' is not defined

In [174]:
output

Name | Summary | Work Experience | Years Of Experience | Skills | Projects | Education | Certifications | Awards and Achievements
Rufus Stewart | Account Executive | Studio Shodwe, Thynk Unlimited, Timmerman Industries | 10 years | Strategy development, Organisation, Public relations, Customer service & relations, Negotiation & networking, Ability to multi-task, Work to deadlines | None | Fauget School of The Arts, Borcelle University | None | None

In [11]:
from langchain.document_loaders import PyPDFLoader
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import json
# Load the PDF
loader1 = PyPDFLoader('media/resumes/Vienna-Modern-Resume-Template.pdf')
pages = loader1.load()

# Get the content of the resume
indiv_resume = pages[0].page_content

# Create the prompt

# Set up the language model

# Get the output


table_template = '''This is a candiates resume {resume} , analyze this and give your output in JSON. For example : ``` "Name": "Alfredo Torres", "Work Experience": [Input the latest company only], "Years Of Experience": "5 years", "Skills": ["Content design", "Speaking", "Content Marketing", "Digital Marketing", "Content editing"], "Projects": ["Salford & co.", "lliceria & co."], "Education": "Only the University Name should be here", "Certifications": "If any certifications done", "Awards and Achievements": None, "AI Review": [ "The candidate has great work experience and professional ethics ]. 

Make sure that all the columns are  present in your response``` '''




promp_analysis_table  = PromptTemplate(template=table_template, input_variables=["resume"])
# Set up the language model
llm = VertexAI(model_name="text-bison@001") # select the model
llm_chain = LLMChain(prompt=promp_analysis_table, llm=llm)
output = llm_chain.run(resume = indiv_resume)
# print(type(output))
# print(output)

# output_list = []
# output_list = json.loads(job_application.job_data())
# output_list.append(output)

# return output


In [14]:
Markdown(output)

{
  "Name": "Janine Nel",
  "Work Experience": [
    {
      "Company": "Engen Oil",
      "Job Title": "Sales Engineer",
      "Dates Employed": "May 2022 - May 2022",
      "Responsibilities": [
        "Recognized and provided current and future customer necessities in areas of filtration and lubrication products and services",
        "Initiated Lubrication Fundamentals classes for employees and customers, which provided education on basic product info, machine elements that need lubrication, application methods, storing

In [12]:
len(output)


530

In [177]:
indiv_resume = pages[0].page_content

table_template = '''Your task is to carefully read through the resume and output a Object with the below given guideline.For example, if youre examining the resume of a software engineer named John Doe, your output should strictly be in Object format.The Object should include the following fields:
    "Name": "", "Work Experience": "","Years Of Experience": "","Skills": [],"Projects": [],"Education": "","Certifications": [],"Awards and Achievements": [], "AI Review" : []
When you encounter a category for which the resume provides no information, please use None as the value.When youve completed your analysis, please provide the extracted details in Object format.
This is the resume of the person:***{resume}***'''



# final = table_template.format(resume=resume)


promp_analysis_table  = PromptTemplate(template=table_template, input_variables=["resume"])
# Set up the language model
llm = VertexAI(model_name="text-bison@001") # select the model
llm_chain = LLMChain(prompt=promp_analysis_table, llm=llm)
new_output = Markdown(llm_chain.run(resume = indiv_resume))
new_output

{
  "Name": "Connor Hamilton",
  "Work Experience": [
    {
      "Company": "Arowwai Industries",
      "Position": "Product Design Manager",
      "Dates": "2010-2012",
      "Description": "Working with the wider development team.
Manage website design, content, and SEO Marketing,
Branding and Logo Design"
    },
    {
      "Company": "Arowwai Industries",
      "Position": "Product Design Manager",
      "Dates": "2

In [178]:
Markdown(new_output)

TypeError: Markdown expects text, not <IPython.core.display.Markdown object>

In [15]:
indiv_resume

'Janine Nel\nSales Engineer\n1515 Pacific Ave, Los Angeles, CA 90291, United States\n3868683442  ·  email@email.com\nLinkedIn, Pinterest, \nResume Templates, \nBuild this template\nPlace of birth\nSan Antonio\nDriving license\nFull\nSkills\nAutoCAD\nIndustry Trends & Sales \nForecasting\nKnowledge of Technical \nDiagrams\nEngineering\nAgile Project Management\nLanguages\nEnglish\nDutch\nHobbies\nPiano, Debate, TriathlonProfile\nDeadline- focused Sales Engineer with six years experience in technical sales, \ncontributing to the 30% growth of income for regional technology solutions \nprovider over five states, producing and assisting long-term contracts with Fortune \n500 clients, and growing Territory sixfold up to $350M.\nEmployment History\nSales Engineer, Engen Oil, Jacksonville\nMay 2022 — May 2022\nPrimary duty was to recognize and provide current and future customer necessities \nin areas of filtration and lubrication products and services.\n•Initiated Lubrication Fundamentals cl

In [16]:
from google.cloud import aiplatform

aiplatform.init(


    credentials='ghc-001-7755ed5582db.json',
)

In [17]:
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project="hackathonvertex", location="us-central1")
parameters = {
    "temperature": 0.2,
    "max_output_tokens": 1024,
    "top_p": 0.8,
    "top_k": 40
}
model = TextGenerationModel.from_pretrained("text-bison@001")
response = model.predict(
    """In this task, you\'ll be analyzing the resume of a job candidate. Your objective is to extract key information from the resume, and evaluate how well the candidate\'s profile aligns with the requirements of the job.

Here is the candidate\'s resume:

***

\'Janine Nel\\nSales Engineer\\n1515 Pacific Ave, Los Angeles, CA 90291, United States\\n3868683442  ·  email@email.com\\nLinkedIn, Pinterest, \\nResume Templates, \\nBuild this template\\nPlace of birth\\nSan Antonio\\nDriving license\\nFull\\nSkills\\nAutoCAD\\nIndustry Trends & Sales \\nForecasting\\nKnowledge of Technical \\nDiagrams\\nEngineering\\nAgile Project Management\\nLanguages\\nEnglish\\nDutch\\nHobbies\\nPiano, Debate, TriathlonProfile\\nDeadline- focused Sales Engineer with six years experience in technical sales, \\ncontributing to the 30% growth of income for regional technology solutions \\nprovider over five states, producing and assisting long-term contracts with Fortune \\n500 clients, and growing Territory sixfold up to $350M.\\nEmployment History\\nSales Engineer, Engen Oil, Jacksonville\\nMay 2022 — May 2022\\nPrimary duty was to recognize and provide current and future customer necessities \\nin areas of filtration and lubrication products and services.\\n•Initiated Lubrication Fundamentals classes for employees and customers, \\nwhich provided education on basic product info, machine elements that \\nneed lubrication, application methods, storing lubricant, and lubricant \\nconservation and handling.\\n•Designed and applied a highly effective industrial filtration program that \\nproduced up to 2,000 gallons of processed oil weekly, which resulted in a \\nconsiderable expense saving for the customer and improved profits for the \\ncompany.\\n•Designed and applied for numerous lubrication programs, providing \\nadditional value-added services and improved profit margins.\\n•Functioned as a consultant on Best-in-Class Lubrication programs.\\nSales Engineer, Quest Medical, Los Angeles\\nJanuary 2019 — April 2021\\nAccountable for day-to-day program management, customer relationships, and \\nestimating the shaping and assembly of medical programs for critical Fortune 500 \\nClients: Pall Life Sciences, Baxter Healthcare, and Quest Medical.\\n•Designed and presented tailored technical solutions to top-3 state \\nautomotive dealers, driving sales from $3M to 11$M over t5 years.\\n•Supervised logistics and Proof of concept pilots worth up to $800,000.\\n•Exceeded yearly quota by 145% by actively outperforming the competition \\nin public tenders and providing 97.5% satisfaction support over the \\nmanaged Territory of current accounts.\\nEducation\\nMasters in Industrial Engineering, Harvard University, Miami\\nJanuary 2019 — May 2022\'


***

Your task is to carefully read through the resume and identify key information about the candidate.
For example, if you\'re examining the resume of a software engineer named John Doe, your output might look like this in JSON format:
The JSON should include the following fields:

    \"Name\": \"John Doe\",
    \"Summary\": \"Software Engineer with 10 years of experience in developing scalable web applications. Proficient in Java, Python, C++.\",
    \"Work Experience\": \"Google Search, Facebook Ads, Uber\",
    \"Years Of Experience\": \"10 years\",
    \"Skills\": [\"Java\", \"Python\", \"C++\"],
    \"Projects\": [\"Google Search\", \"Facebook Ads\", \"Uber\"],
    \"Education\": \"Stanford University\",
    \"Certifications\": [\"AWS Certified Solutions Architect - Associate\", \"Google Cloud Certified Professional Cloud Architect\"],
    \"Awards and Achievements\": [\"Google Founders\' Award\", \"ACM Grace Hopper Award\"]


When you encounter a category for which the resume provides no information, please use None as the value.

The quality of your work will be assessed based on:

The completeness and accuracy of the information you extract from the resume.
The relevance of the extracted information to the job posting.
The correctness of the JSON format in your output.
Now, here\'s your question:


When you\'ve completed your analysis, please provide the extracted details in JSON format.
\"\"\"""",
    **parameters
)
print(f"Response from Model: {response.text}")

RetryError: Deadline of 120.0s exceeded while calling target function, last exception: 503 Getting metadata from plugin failed with error: 'str' object has no attribute 'before_request'